<a href="https://colab.research.google.com/github/Fra3005/Computer-Vision/blob/main/CNN_%26_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 5.0 MB/s 


In [3]:
from tensorflow.keras import datasets, layers, models


#Split dataset into train and test

In [111]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()


In [112]:
print(train_images.shape)
print(test_images.shape)

train_images = train_images / 255.0
test_images = test_images / 255.0

(50000, 32, 32, 3)
(10000, 32, 32, 3)


#Create model for CNN

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(10, activation='softmax'))

model.summary() #displays the network architecture

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          8256      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 64)         0

#Fit CNN model whit a EarlyStopping Callback

In [7]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=2)


model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1563/1563 [==============================] - 63s 39ms/step - loss: 1.5123 - accuracy: 0.4531
Epoch 2/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.1805 - accuracy: 0.5776
Epoch 3/10
1563/1563 [==============================] - 62s 39ms/step - loss: 1.0517 - accuracy: 0.6268
Epoch 4/10
1563/1563 [==============================] - 62s 40ms/step - loss: 0.9773 - accuracy: 0.6544
Epoch 5/10
1563/1563 [==============================] - 62s 39ms/step - loss: 0.9111 - accuracy: 0.6774
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 0.8666 - accuracy: 0.6962
Epoch 7/10
1563/1563 [==============================] - 63s 40ms/step - loss: 0.8261 - accuracy: 0.7108
Epoch 8/10
1563/1563 [==============================] - 62s 40ms/step - loss: 0.7964 - accuracy: 0.7180
Epoch 9/10
1563/1563 [==============================] - 67s 43ms/step - loss: 0.7631 - accuracy: 0.7287
Epoch 10/10
1563/1563 [==============================] - 65s 42m

#Get the accuracy of the model

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy=',test_acc)

313/313 [==============================] - 4s 13ms/step - loss: 0.7695 - accuracy: 0.7331
Test accuracy= 0.7330999970436096


In [68]:
# get predictions on the test set
y_hat = model.predict(test_images)

#Apply Hog filter to Training and Test sets

In [113]:
from skimage.feature import hog
from cv2 import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from skimage.color import rgb2gray


train_im_gray = [rgb2gray(instance) for instance in train_images]
test_im_gray = [rgb2gray(instance) for instance in test_images]

train_im = [hog(img, cells_per_block=(2, 2)) for img in train_im_gray]
test_im = [hog(img, cells_per_block=(2, 2)) for img in test_im_gray]



In [114]:
train_im = np.array(train_im)
test_im = np.array(test_im)

#Apply a Standardization to the set

In [115]:
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier 
scaler = StandardScaler()  
scaler.fit(train_im)
X_train = scaler.transform(train_im)  
X_test = scaler.transform(test_im) 


In [116]:
from tensorflow.keras.utils import to_categorical


train_labels = train_labels.flatten() 
test_label = test_labels.flatten()


In [117]:
clf = MLPClassifier(
                    hidden_layer_sizes=(5, 2),
                    random_state=1,
                    verbose=True,
                    learning_rate_init=0.02)

In [118]:
clf.fit(X_train, train_labels)  

Iteration 1, loss = 1.92482030
Iteration 2, loss = 1.74765556
Iteration 3, loss = 1.72699343
Iteration 4, loss = 1.71648775
Iteration 5, loss = 1.70745275
Iteration 6, loss = 1.70481660
Iteration 7, loss = 1.70038234
Iteration 8, loss = 1.69592765
Iteration 9, loss = 1.69500564
Iteration 10, loss = 1.69362580
Iteration 11, loss = 1.68922003
Iteration 12, loss = 1.69206943
Iteration 13, loss = 1.68787386
Iteration 14, loss = 1.68736150
Iteration 15, loss = 1.68411526
Iteration 16, loss = 1.68702400
Iteration 17, loss = 1.67979112
Iteration 18, loss = 1.68003336
Iteration 19, loss = 1.67672690
Iteration 20, loss = 1.67842490
Iteration 21, loss = 1.67744601
Iteration 22, loss = 1.67431566
Iteration 23, loss = 1.67818904
Iteration 24, loss = 1.67632249
Iteration 25, loss = 1.67223746
Iteration 26, loss = 1.67556354
Iteration 27, loss = 1.67465350
Iteration 28, loss = 1.67521324
Iteration 29, loss = 1.67342478
Iteration 30, loss = 1.67000999
Iteration 31, loss = 1.67184969
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(5, 2), learning_rate_init=0.02,
              random_state=1, verbose=True)